In [ ]:
%mamba install sparrow

In [ ]:
#include <sparrow.hpp>
namespace sp = sparrow;
#include <iostream>
#include <iomanip>

#include <sparrow/config/sparrow_version.hpp>
std::cout << "Sparrow version: " <<  sparrow::SPARROW_VERSION_MAJOR << "." << sparrow::SPARROW_VERSION_MINOR << "." << sparrow::SPARROW_VERSION_PATCH;

# sparrow::nullable

In [ ]:
sp::nullable<int> n1;
std::cout << (n1.has_value() ? "true" : "false") << std::endl;

In [ ]:
sp::nullable<int> n2 = 42;
std::cout << (n2.has_value() ? "true" : "false") << std::endl;
std::cout << n2.value() << std::endl;

In [ ]:
sp::nullable<int> n3 = sp::make_nullable(7, false);
std::cout << (n3.has_value() ? "true" : "false") << std::endl;
std::cout << n3.get() << std::endl;

# Typed Array Construction

In [ ]:
const std::vector<int32_t> values = { 1, 2, 3, 4, 5 };
const std::vector<bool> validity { true, true, false, true, true };

In [ ]:
sp::primitive_array<int32_t> ar2{values, validity, "my_array"};

In [ ]:
sp::u8_buffer<int32_t> buffer{values};

# Typed Array Access

In [ ]:
std::cout << "name: "               << ar2.name().value()  << std::endl;
std::cout << "size: "               << ar2.size()          << std::endl;
std::cout << "front: "              << ar2.front().value() << std::endl;
std::cout << "back: "               << ar2.back().value()  << std::endl;
std::cout << "ar[4].has_value(): "  << ar2[4].has_value()  << std::endl;
std::cout << "ar[4].get(): "        << ar2[4].get()        << std::endl;
std::cout << "ar[4].value_or(42): " << ar2[4].value_or(42) << std::endl;

In [ ]:
for(const auto& nullable_value : ar2)
{
    std::cout<< (nullable_value.has_value() ? std::to_string(nullable_value.value()) : "null") << ", ";
}

In [ ]:
for(const auto& value : ar2.values())
{
    std::cout<< value << ", ";
}

In [ ]:
for(const auto& validity : ar2.bitmap())
{
    std::cout<< (validity ? "true" : "false") << ", ";
}

# Typed Array Mutability

In [ ]:
ar2[3] = 0;
std::cout<< ar2[3].value();

In [ ]:
ar2[4] = sp::make_nullable(7, false);
std::cout<< (ar2[4].has_value() ? "true" : "false") << std::endl;
std::cout<< ar2[4].get() << std::endl;
ar2[4].null_flag() = true;
std::cout<< (ar2[4].has_value() ? "true" : "false") << std::endl;
std::cout<< ar2[4].get() << std::endl;

In [ ]:
try {ar2.at(99); }// throws
catch(...) {
    std::cerr<< "Caught!";
}

In [ ]:
ar2.push_back(sp::make_nullable(111, false));
std::cout<< ar2.back().has_value() << std::endl;
std::cout<< ar2.back().get() << std::endl;

In [ ]:
ar2.pop_back();
std::cout<< ar2.back().get();

In [ ]:
ar2.insert(ar2.cbegin() + 2, sp::make_nullable(222, true));
for(const auto& nullable_value : ar2)
{
    std::cout<< (nullable_value.has_value() ? std::to_string(nullable_value.value()) : "null") << ", ";
}

In [ ]:
ar2.resize(20, sp::make_nullable(333, true)); 
for(const auto& nullable_value : ar2)
{
    std::cout<< (nullable_value.has_value() ? std::to_string(nullable_value.value()) : "null") << ", ";
}


In [ ]:
ar2.zero_null_values();
for(const auto& nullable_value : ar2)
{
    std::cout << std::setw(5) << (nullable_value.has_value() ? std::to_string(nullable_value.value()) : "null") << ", ";
}
std::cout << std::endl;
for(const auto& value : ar2.values())
{
     std::cout << std::setw(5) << value << ", ";
}

# Slicing

In [ ]:
sp::primitive_array<int32_t> ar_slice = ar2.slice(3, 5);
for(const auto& value : ar_slice.values())
{
     std::cout << value << ", ";
}

In [ ]:
sp::primitive_array<int32_t> ar_slice_view = ar2.slice_view(3, 5);

In [ ]:
for(const auto& value : ar_slice.values())
{
     std::cout << value << ", ";
}

# Interoperability with Arrow C Data Interface

In [ ]:
// ArrowArray* array_ptr = get_arrow_array(ar2);
// ArrowSchema* schema_ptr = get_arrow_schema(ar2);
auto [arrow_array_ptr, arrow_schema_ptr] = sp::get_arrow_structures(ar2);

// ArrowArray array = extract_arrow_array(arr);
// ArrowSchema schema = extract_arrow_schema(arr);
auto [arrow_array, arrow_schema] = sp::extract_arrow_structures(std::move(ar2));

# Untyped Arrays

In [ ]:
sp::array ar(std::move(arrow_array), std::move(arrow_schema));

ar.visit([]<class T>(const T& typed_ar)
{
    if constexpr (sp::is_primitive_array_v<T>)
    {
        for(const auto& value : typed_ar.values())
        {
             std::cout << value << ", ";
        }
    }
    // else if constexpr ...
});

# Record Batch

In [ ]:
#include <sparrow/record_batch.hpp>
sp::primitive_array<std::uint16_t> ar_int(
    std::vector<std::uint16_t>{ 1, 2, 3, 4, 5 },
    sp::validity_bitmap{sp::validity_bitmap::default_allocator()},
    "my_primitives");

sp::string_array ar_str(
    std::vector<std::string>{ "one", "two", "three", "four", "five" },
    sp::validity_bitmap{sp::validity_bitmap::default_allocator()},
    "my_strings");

std::vector<sp::array> arr_list = {sp::array(std::move(ar_int)), sp::array(std::move(ar_str))};

sp::record_batch rb(std::move(arr_list), "my_record_batch");

std::cout << "Columns count: " << rb.nb_columns() << std::endl;
std::cout << "Rows count: " << rb.nb_rows() << std::endl;

for(const auto& column : rb.columns())
{
    column.visit([]<class T>(const T& typed_ar)
    {
        std::cout << typed_ar.name().value() << ": ";
        if constexpr (sp::is_primitive_array_v<T> || sp::is_string_array_v<T>)
        {
            for(const auto& value : typed_ar.values())
            {
                 std::cout << value << ", ";
            }
        }   
    });
    std::cout << std::endl;
}